# Federated learning

In [1]:
# Pobierz biblioteki
!pip install -q flwr[simulation] tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.4 MB/s eta 0:0

In [3]:
# Importuj biblioteki
import os
from functools import partial

import flwr as fl
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import Model

In [4]:
from platform import python_version
print(f'python: {python_version()}')
print(f"\nflwr: {fl.__version__}")
print(f"numpy: {np.__version__}")
print(f"tensorflow: {tf.__version__}")

python: 3.9.16

flwr: 1.3.0
numpy: 1.22.4
tensorflow: 2.12.0


In [5]:
# Użyjmy tylko CPU
tf.config.set_visible_devices([], 'GPU')

In [7]:
# Załadujmy zbiór dancyh MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Przeskalujmy dane do [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Dodajmy dodatkowy wymiar to danych (wymagane przez Conv2D)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

11490434/11490434 [==============================] - 1s 0us/step


In [8]:
# Podzielmy dane na 5 klientów
NUM_CLIENTS = 5
x_split = np.split(x_train, NUM_CLIENTS)
y_split = np.split(y_train, NUM_CLIENTS)

# Stwórzmy mapę cid (id klient) do danych
# Dla danych treningowych i testowych
num_data_in_split = x_split[0].shape[0]
train_split = 0.8
x_trains, y_trains, x_tests, y_tests = {}, {}, {}, {}
for idx, (client_x, client_y) in enumerate(zip(x_split, y_split)):
   train_end_idx = int(0.8 * num_data_in_split)
   x_trains[str(idx)] = client_x[:train_end_idx]
   y_trains[str(idx)] = client_y[:train_end_idx]
   x_tests[str(idx)] = client_x[train_end_idx:]
   y_tests[str(idx)] = client_y[train_end_idx:]


In [29]:
for i in ['0','1','2','3','4']:
  print(f"x_trains['{i}'].shape = {x_trains[i].shape}; \
  x_tests['{i}'].shape = {x_tests[i].shape};")

x_trains['0'].shape = (9600, 28, 28, 1);   x_tests['0'].shape = (2400, 28, 28, 1);
x_trains['1'].shape = (9600, 28, 28, 1);   x_tests['1'].shape = (2400, 28, 28, 1);
x_trains['2'].shape = (9600, 28, 28, 1);   x_tests['2'].shape = (2400, 28, 28, 1);
x_trains['3'].shape = (9600, 28, 28, 1);   x_tests['3'].shape = (2400, 28, 28, 1);
x_trains['4'].shape = (9600, 28, 28, 1);   x_tests['4'].shape = (2400, 28, 28, 1);


In [32]:
class CNN(Model):
   def __init__(self):
       super(CNN, self).__init__()
       self.conv1 = Conv2D(32, (3, 3), activation='relu')
       self.pool1 = MaxPooling2D((2, 2))
       self.conv2 = Conv2D(64, (3, 3), activation='relu')
       self.pool2 = MaxPooling2D((2, 2))
       self.flatten = Flatten()
       self.dense1 = Dense(128, activation='relu')
       self.dropout = Dropout(0.5)
       self.dense2 = Dense(10, activation='softmax')


   def call(self, inputs):
       x = self.conv1(inputs)
       x = self.pool1(x)
       x = self.conv2(x)
       x = self.pool2(x)
       x = self.flatten(x)
       x = self.dense1(x)
       x = self.dropout(x)
       return self.dense2(x)


In [33]:
# Zdefiniujmy Klienta Flower
class FlowerClient(fl.client.NumPyClient):
   def __init__(self, model, X_train, y_train, X_test, y_test):
       self.model = model
       self.model.build((32, 28, 28, 1))
       self.X_train = X_train
       self.y_train = y_train
       self.X_test = X_test
       self.y_test = y_test


   def get_parameters(self, config):
       return self.model.get_weights()


   def fit(self, parameters, config):
       self.model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
       self.model.set_weights(parameters)
       self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
       return self.model.get_weights(), len(x_train), {}


   def evaluate(self, parameters, config):
       self.model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
       self.model.set_weights(parameters)
       loss, accuracy = self.model.evaluate(self.X_test, self.y_test, batch_size=32, verbose=0)
       return loss, len(self.X_test), {"accuracy": accuracy}

In [34]:
# Pełna funcja tworząca jednego clienta Flower
def create_client(
   cid, model_class, x_trains, y_trains, x_tests, y_tests
) -> FlowerClient:
   """Create a Flower client representing a single organization."""
   model = model_class()
   # Create a  single Flower client representing a single organization
   return FlowerClient(model, x_trains[cid], y_trains[cid], x_tests[cid], y_tests[cid])


# Funkcja tworząca jednego klienta Flower z tylko jednym argumentem - cid
# Reszta argumentów jest stała
client_fnc = partial(
   create_client,
   model_class=CNN,
   x_trains=x_trains,
   y_trains=y_trains,
   x_tests=x_tests,
   y_tests=y_tests,
)


In [35]:
# Uśrednianie metryk
def weighted_average(metrics):
   print(metrics)
   accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
   examples = [num_examples for num_examples, _ in metrics]
   return {"accuracy": int(sum(accuracies)) / int(sum(examples))}


# Stwórzmy strategię FedAvg
strategy = fl.server.strategy.FedAvg(
   fraction_fit=1.0,  # Samplujmy 100% dostępnych klientów na trening
   fraction_evaluate=1.0,  # Samplujmy 100% dostępnych klientów na evaluację
   min_fit_clients=5,  # Nie samplujmy mniej niż 5 klientów na trening
   min_evaluate_clients=5,  #Nie samplujmy mniej niż 5 klientów na evaluację
   min_available_clients=5,  # Poczekaj aż 5 klientów jest dostępnych
   evaluate_metrics_aggregation_fn=weighted_average, # Uśrednianie metryk
)


In [14]:
fl.simulation.start_simulation(
   client_fn=client_fnc,
   num_clients=NUM_CLIENTS,
   config=fl.server.ServerConfig(num_rounds=10),
   strategy=strategy,
   client_resources={"num_cpus": 1, "num_gpus": 0},
   ray_init_args={
       "num_cpus": 1,
       "num_gpus": 0,
       "_system_config": {"automatic_object_spilling_enabled": False},
   },
)


INFO flwr 2023-04-03 10:41:34,645 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-04-03 10:41:39,090	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-03 10:41:40,961 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 7902727374.0, 'object_store_memory': 3951363686.0, 'CPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7902727374.0, 'object_store_memory': 3951363686.0, 'CPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-04-03 10:41:40,982 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-03 10:41:40,991 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(launch_and_get_parame

[(2400, {'accuracy': 0.9445833563804626}), (2400, {'accuracy': 0.9558333158493042}), (2400, {'accuracy': 0.9704166650772095}), (2400, {'accuracy': 0.9641666412353516}), (2400, {'accuracy': 0.95291668176651})]


(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90f8d820>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bound method CNN.call of <__main__.CNN object at 0x7fda90f8d820>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(launch_and_fit pid=1897) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda95ffbb50>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bo

[(2400, {'accuracy': 0.9754166603088379}), (2400, {'accuracy': 0.9800000190734863}), (2400, {'accuracy': 0.972083330154419}), (2400, {'accuracy': 0.9670833349227905}), (2400, {'accuracy': 0.9741666913032532})]


(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda8ff2bdf0>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bound method CNN.call of <__main__.CNN object at 0x7fda8ff2bdf0>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(launch_and_fit pid=1897) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90f93b50>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bo

...

(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90d62670>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bound method CNN.call of <__main__.CNN object at 0x7fda90d62670>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(launch_and_fit pid=1897) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90cd95b0>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bo

[(2400, {'accuracy': 0.9900000095367432}), (2400, {'accuracy': 0.9891666769981384}), (2400, {'accuracy': 0.9816666841506958}), (2400, {'accuracy': 0.9916666746139526}), (2400, {'accuracy': 0.987500011920929})]


(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90d86400>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bound method CNN.call of <__main__.CNN object at 0x7fda90d86400>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
(launch_and_fit pid=1897) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(launch_and_fit pid=1897) WARNING:tensorflow:AutoGraph could not transform <bound method CNN.call of <__main__.CNN object at 0x7fda90e932e0>> and will run it as-is.
(launch_and_fit pid=1897) Cause: Unable to locate the source code of <bo

[(2400, {'accuracy': 0.9904166460037231}), (2400, {'accuracy': 0.9820833206176758}), (2400, {'accuracy': 0.9879166483879089}), (2400, {'accuracy': 0.9908333420753479}), (2400, {'accuracy': 0.9904166460037231})]


History (loss, distributed):
	round 1: 0.22658251225948334
	round 2: 0.08721537441015244
	round 3: 0.06703172698616981
	round 4: 0.058258906751871106
	round 5: 0.0516501396894455
	round 6: 0.04955654218792915
	round 7: 0.04499546885490417
	round 8: 0.043798650801181796
	round 9: 0.04142874479293823
	round 10: 0.03940202072262764
History (metrics, distributed):
{'accuracy': [(1, 0.9575), (2, 0.97375), (3, 0.9805), (4, 0.9831666666666666), (5, 0.9843333333333333), (6, 0.9840833333333333), (7, 0.9860833333333333), (8, 0.9860833333333333), (9, 0.988), (10, 0.98825)]}